<h3> Model Registry Initiation </h3>

In [8]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow


<h3> Set mlflow tracking uri and experiment </h3>

In [9]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("kkbox-churn-prediction3")

<Experiment: artifact_location=('/Users/sarveshthakur/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp-practice/capstone project/mlruns/3'), creation_time=1692204804852, experiment_id='3', last_update_time=1692204804852, lifecycle_stage='active', name='kkbox-churn-prediction3', tags={}>

In [10]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
# client = MlflowClient("http://127.0.0.1:5000")

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
experiment = client.get_experiment_by_name("kkbox-churn-prediction3")

In [12]:
top_n = 5

runs = client.search_runs(
experiment_ids=experiment.experiment_id,
run_view_type=ViewType.ACTIVE_ONLY,
max_results=top_n,
order_by=["metrics.roc_auc_score ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, roc_auc_score: {run.data.metrics['roc_auc_score']:.4f}")





run id: f8173dd34507482e9258a0b699b1e02c, roc_auc_score: 0.5000
run id: 8eb6c1d188c7441c900ee29f0f4917ec, roc_auc_score: 0.6298
run id: e700f5491b3242798ab1f45c60742358, roc_auc_score: 0.6298
run id: eff24819b0d443a48315955e1123fc94, roc_auc_score: 0.6298
run id: c4024f3eec4c49a694177bcda23304ab, roc_auc_score: 0.6298


<h3> Register model </h3>

In [13]:
run_id = 'c4024f3eec4c49a694177bcda23304ab'
model_uri = f'runs:/{run_id}/model'

mlflow.register_model(model_uri=model_uri, name="kkbox-churn-prediction3")

Registered model 'kkbox-churn-prediction3' already exists. Creating a new version of this model...
2023/08/16 15:19:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: kkbox-churn-prediction3, version 2


Created version '2' of model 'kkbox-churn-prediction3'.


<ModelVersion: aliases=[], creation_timestamp=1692213580890, current_stage='None', description=None, last_updated_timestamp=1692213580890, name='kkbox-churn-prediction3', run_id='c4024f3eec4c49a694177bcda23304ab', run_link=None, source=('/Users/sarveshthakur/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp-practice/capstone '
 'project/mlruns/3/c4024f3eec4c49a694177bcda23304ab/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=2>

<h3> Promote model to staging </h3>

In [14]:
model_name = "kkbox-churn-prediction3"
latest_versions = client.get_latest_versions(name=model_name)

In [15]:
for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: None


In [16]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Staging",
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1692210263558, current_stage='Staging', description=None, last_updated_timestamp=1692213585714, name='kkbox-churn-prediction3', run_id='c4024f3eec4c49a694177bcda23304ab', run_link=None, source=('/Users/sarveshthakur/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp-practice/capstone '
 'project/mlruns/3/c4024f3eec4c49a694177bcda23304ab/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>

In [17]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1692210263558, current_stage='Production', description=None, last_updated_timestamp=1692213586442, name='kkbox-churn-prediction3', run_id='c4024f3eec4c49a694177bcda23304ab', run_link=None, source=('/Users/sarveshthakur/Documents/MLOps '
 'Zoomcamp/mlops-zoomcamp-practice/capstone '
 'project/mlruns/3/c4024f3eec4c49a694177bcda23304ab/artifacts/model'), status='READY', status_message=None, tags={}, user_id=None, version=1>